# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
df=pd.read_csv("output.csv")
df.head()
print(df["humidity"])

0      40
1      71
2      77
3      91
4      66
       ..
470    65
471    42
472    75
473    90
474    68
Name: humidity, Length: 475, dtype: int64


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["latitude", "longtitude"]].astype(float)
humid=df["humidity"].astype(float)

# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#temp is in Kelvnis 294.26=70 F
hotel_df=df.loc[(df["temperature"]>294.26)&(df["temperature"]<299.817)&(df["wind_speed"]<10)&(df["cloudiness"]<30)]
hotel_df.head()

,Unnamed: 0,city_name,temperature,humidity,wind_speed,cloudiness,latitude,longtitude
11,11,Jiwani,298.71,24,8.85,0,25.0500,61.7417
18,18,Şalālah,297.15,38,1.50,0,17.0151,54.0924
39,39,Nizwá,295.50,17,3.28,0,22.9333,57.5333
112,112,Baswa,297.67,20,0.40,6,27.1500,76.5833
117,117,Shujālpur,298.36,76,1.72,29,23.4000,76.7167


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
lati=hotel_df["latitude"].to_list()
long=hotel_df["longtitude"].to_list()

hotel_names=[]
hotel_lat=[]
hotel_long=[]
print(len(lati))

for x in range(len(lati)):
    try:
        target_radius=5000
        taget_type="hotel"
        target_coordinates=f"{lati[x]}, {long[x]}"
        params = {
        "location":target_coordinates,
        "radius": target_radius,
        "key": g_key}
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params=params)
        hotel_data = response.json()
#print(json.dumps(hotel_data, indent=4, sort_keys=True))
        hotel_names.append(hotel_data["results"][0]["name"])
        hotel_lat.append(hotel_data["results"][0]["geometry"]["location"]["lat"])
        hotel_long.append(hotel_data["results"][0]["geometry"]["location"]["lng"])
    except IndexError:
        print(f"No hotel in {x}")

32


In [11]:
datadict={"Hotel name":hotel_names,"Latitude":hotel_lat,"Longtitude":hotel_long}
hotel_df1=pd.DataFrame(datadict)

In [14]:

hotel_df1.to_csv("output_hotel.csv")
hotel_df1.head()

,Hotel name,Latitude,Longtitude
0,Jīwani,25.053751,61.770729
1,Salalah,17.019384,54.110750
2,Nizwa,22.917103,57.536292
3,Baswa,27.141904,76.589277
4,Shujalpur,23.388202,76.716336


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df1[["Latitude", "Longtitude"]]

KeyError: 'Hotel name'

# Add marker layer ontop of heat map


# Display figure
